# writing 'command line' Python scripts
- scripts can query a user, but they usually run in 'batch' mode with no interaction
- very important Python skill, fortunately pretty easy
- python is much nicer than 'bash'
- great for automating repetitive tasks
- if writing a complex script, often easier to write the bulk of the code as functions,
which can be tested in an IDE, or a notebook. then just call the functions based on the command kube args to the script
- on command line can do'python script.py'
- on mac/linux can make script itself directly executable
    - put magic line:
    - ```#!/usr/bin/env python``` 
    - at top of file to invoke python
    - must give file 'execute' permission


# sys module
- basic interface for command line programs

# sys.argv 
- variable set to command line arguments, like 'main' in C and Java
- no argc variable - just do len(sys.argv)

# sys.exit(status) 
- exit python and return status
- status is a BYTE, 0-255 only
- in scripts, main method for signalling script success or failure
- return of 0 means things went ok, other values indicate error
- in mac/linux, can see exit code by doing
    - echo $?

# 'input' function
- most scripts run in a 'batch mode' w/o interaction
- scripts usually controlled by 'command line args'
- but, sometimes a script may want to query the user
    - 'input()' will read a line of text from keyboard
    - 'input('prompt') will first print 'prompt' string, then wait for input
    - works in scripts and notebooks

In [1]:
input("Type a line: ")

Type a line: bob


'bob'

# tqdm
- nice progress bar
- works in notebooks and scripts

In [2]:
from tqdm import tqdm
import time

for j in tqdm(range(20)):
        time.sleep(1)

100%|██████████| 20/20 [00:20<00:00,  1.00s/it]


# argparse module
- elaborate command line args parser
- [argparse reference](https://docs.python.org/3.5/library/argparse.html)
- [argparse tutorial](https://docs.python.org/3.5/howto/argparse.html) - easier to read
- takes a little effort to set it up, but provides alot of functionality. handles many error conditions automatically
- will do sys.exit(1) on invalid args
- like 'getopt' in C


In [ ]:
# needs to run from cmd line
# contents of argparse-example

'''
#!/usr/bin/env python

# run as python script

import sys
import argparse

parser = argparse.ArgumentParser()

# required positional arg
parser.add_argument("arg1", type=int,
                    help="an int")

# 2nd required positional arg
parser.add_argument("arg2", type=str,
                    help="a pathname")

# optional '-' flag with no arg
parser.add_argument("-v", "--verbose", help="increase output verbosity",
                    action="store_true")

# optional '-' flag no arg
parser.add_argument("-q", "--query", help="query for exit value",
			 action='store_true')

# optional '-' flag with required arg
parser.add_argument("-e", "--exit", help="specify exit value as arg",
			 type=int)


# parsed args - will automatically print errors and abort on bad args
args = parser.parse_args()

# if we got here, the parse was happy
print('arg1=', args.arg1)
print('arg2=', args.arg2)

# args.verbose will = None if no arg
if args.verbose:
    print("verbosity turned on")


# note exit val is one byte, 0-255
# could check and complain if given
# bogus value

if args.exit:
   print("exit with:", args.exit)
   # in bash, print with:  echo $?
   sys.exit(args.exit)
   print("won't get here")

if args.query:
   # demand input from the user
   es = input('specify exit value: ')
   e = int(es)
   sys.exit(e)


'''
None

In [ ]:
# sample calls

'''
# use python to invoke script
# bad args, doesn't work

python cmdline.py 
usage: cmdline.py [-h] [-v] [-e EXIT] intarg1 arg2
cmdline.py: error: the following arguments are required: intarg1, arg2


# invoke script directly. ask for help with '-h'

notebooks@larrys-MBP$ cmdline.py -h
usage: cmdline.py [-h] [-v] [-q] [-e EXIT] arg1 arg2

positional arguments:
  arg1                  an int
  arg2                  a pathanme

optional arguments:
  -h, --help            show this help message and exit
  -v, --verbose         increase output verbosity
  -q, --query           query for exit value
  -e EXIT, --exit EXIT  specify exit value as arg


# fails, no args

notebooks@larrys-MBP$ cmdline.py
usage: cmdline.py [-h] [-v] [-e EXIT] intarg1 arg2
cmdline.py: error: the following arguments are required: intarg1, arg2


# doesn't work, first arg must be an int


notebooks@larrys-MBP$ cmdline.py foo
usage: cmdline.py [-h] [-v] [-e EXIT] intarg1 arg2
cmdline.py: error: argument intarg1: invalid int value: 'foo'


# doesn't work, missing 2nd arg


notebooks@larrys-MBP$ cmdline.py 1234
usage: cmdline.py [-h] [-v] [-e EXIT] intarg1 arg2
cmdline.py: error: the following arguments are required: arg2


# finally got it right!

notebooks@larrys-MBP$ cmdline.py 1234 bar
Namespace(arg2='bar', exit=None, intarg1=1234, verbose=False)
intarg1= 1234
arg2= bar


# added optional '-v' verbose flag

notebooks@larrys-MBP$ cmdline.py 1234 bar -v
Namespace(arg2='bar', exit=None, intarg1=1234, verbose=True)
intarg1= 1234
arg2= bar
verbosity turned on

# added optional '--exit' flag, which takes an arg

notebooks@larrys-MBP$ cmdline.py 1234 bar --exit 29
Namespace(arg2='bar', exit=29, intarg1=1234, verbose=False)
intarg1= 1234
arg2= bar
exit with: 29
notebooks@larrys-MBP$ echo $?
29



'''

# alternative module - use 'click'
- arg parsing specified with decorators
- @click.argument - like python positional args
- @click.option - like python optional keyword args
    - options normally take an args, like '--exit'
    - is_flag = True - optional does not take an arg
- tends to be simpler than argparse

In [ ]:
import time
import click
import sys
from tqdm import tqdm

@click.command()
@click.argument('arg1')
@click.argument('arg2')
@click.option('--loops', default=0, help='Loop with progress bar')
@click.option('--exit', default=0, help='Specifiy exit code')
@click.option('--verbose', is_flag=True, help='Print sys info')
@click.option('--query', is_flag=True, help='Prompt for exit code')


def hello(arg1, arg2, exit=0, query=False, verbose=False, loops = 0):
    """Simple script that plays with click."""
    if verbose:
        print(f'sys.version={sys.version}')
        print(f'arg1={arg1}, arg2={arg2}')
    if loops:
        for j in tqdm(range(loops)):
            time.sleep(1)
    if exit:
        sys.exit(exit)

    if query:
        # demand input from the user
        es = input('specify exit value: ')
        e = int(es)
        sys.exit(e)
        

hello()


```
week6$ click-example --help
Usage: click-example [OPTIONS] ARG1 ARG2

  Simple script that plays with click.

Options:
  --loops INTEGER  Loop with progress bar
  --exit INTEGER   Specifiy exit code
  --verbose        Print sys info
  --query          Prompt for exit code
  --help           Show this message and exit.
week6$ click-example first 2
week6$ click-example --verbose first 2
3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
arg1=first, arg2=2
week6$ click-example --loops 4 1 2
100%|█████████████████████████████████████████████| 4/4 [00:04<00:00,  1.00s/it]
week6$ click-example --exit 35 1 2
week6$ echo $?
35
week6$ click-example --query 1 2
specify exit value: 11
week6$ echo $?
11


```